<a href="https://colab.research.google.com/github/fernando2393/DT2119-Final-Project/blob/Andres/MusicGenreClassification_Andres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
from google.colab import files
files.upload()

Saving constants.py to constants (1).py
Saving features.py to features (1).py
Saving loader.py to loader (1).py


{'constants.py': b'# Run locally\r\n# DATASETS = "Datasets/fma_small/"\r\n# METADATA = "Datasets/fma_metadata/"\r\n# MFCC = "Datasets/mfcc.csv"\r\n# SPECTROGRAM = "Datasets/spectrogram.csv"\r\n\r\n# # Run on Google Colab\r\nDATASETS = "/content/drive/My Drive/Datasets/fma_small/"\r\nMETADATA = "/content/drive/My Drive/Datasets/fma_metadata/"\r\nMFCC = "/content/drive/My Drive/Datasets/mfcc.csv"\r\nSPECTROGRAM = "/content/drive/My Drive/Datasets/spectrogram.csv"\r\n',
 'features.py': b'import pandas as pd\r\nimport sklearn\r\nimport librosa.display\r\nimport librosa\r\nimport os\r\nfrom tqdm import tqdm\r\nimport warnings\r\nimport constants as cts\r\nimport numpy as np\r\n\r\n\r\ndef compute_feature(mode, filepath):\r\n    """\r\n    This method loads the a music track and computes the mfcc.\r\n    :param mode: allows to get either spectrogram or mfcc.\r\n    :param filepath: music track.\r\n    :return mfcc of the data track.\r\n    """\r\n    if mode == \'spectrogram\':\r\n        y,

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, ZeroPadding2D, ELU, Reshape, GRU
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
import loader
(x_train, y_train), (x_val, y_val), (x_test, y_test) = loader.get_train_val_test(mode='spectrogram')
print("There are the following classes:")
classes = set(y_train.tolist()) & set(y_val.tolist()) & set(y_test.tolist())
print(classes)

Calculating spectrogram...
There are the following classes:
{'Experimental', 'Electronic', 'Rock', 'International', 'Folk', 'Instrumental', 'Hip-Hop', 'Pop'}


In [0]:
# Remove small samples
x_train = np.delete(x_train, [3495, 3496, 3497])
y_train = np.delete(y_train, [3495, 3496, 3497])
x_train = np.rollaxis(np.dstack(x_train), -1)
x_val = np.rollaxis(np.dstack(x_val), -1)
x_test = np.rollaxis(np.dstack(x_test), -1)
x_train = np.expand_dims(x_train, axis=3)
x_val = np.expand_dims(x_val, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [0]:
# One-hot encoding of classes
dict_labels = {'Electronic': 0, 'Experimental': 1, 'Folk': 2, 'Hip-Hop': 3,
                'Instrumental': 4, 'International': 5, 'Pop': 6, 'Rock': 7}
y_train = [dict_labels[y_train[i]] for i in range(y_train.shape[0])]
y_val = [dict_labels[y_val[i]] for i in range(y_val.shape[0])]
y_test = [dict_labels[y_test[i]] for i in range(y_test.shape[0])]
y_train = to_categorical(y_train, num_classes=8)
y_val = to_categorical(y_val, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

In [0]:
class Classifier:

    @staticmethod
    def build(inputs, classes):

        x = Conv2D(16, (3, 3), strides=(1, 1))(inputs)
        x = tf.nn.relu(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(x)

        x = Conv2D(32, (3, 3), strides=(1, 1))(x)
        x = tf.nn.relu(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(x)

        x = Conv2D(64, (3, 3), strides=(1, 1))(x)
        x = tf.nn.relu(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(x)
        x = Dropout(rate=0.1, trainable=True)(x)

        x = Flatten()(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(rate=0.3, trainable=True)(x)
        outputs = Dense(classes, activation='softmax')(x)

        return tf.keras.Model(inputs=inputs, outputs=outputs)

In [8]:
# Training parameters
  epochs = 25  # Train for 25 epochs
  lr = 0.005  # Initial learning rate
  batch_size = 16

  inputs = tf.keras.Input(shape=x_train.shape[1:])
  model = Classifier.build(inputs, classes=8)
  print("Summary:")
  print(model.summary())

  opt = Adam(lr=lr, decay=lr / epochs)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
  # Callbacks: early stopping and checkpoint
  early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1,
                                  patience=10,
                                  mode='max',
                                  restore_best_weights=True)
  filepath = "weights.{epoch:02d}-{val_accuracy:.2f}.hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                                save_best_only=True, mode='max')
  callbacks_list = [early_stopping, checkpoint]
  history = model.fit(x_train, y_train, batch_size=batch_size,
                      validation_data=(x_val, y_val),
                      steps_per_epoch=len(x_train) // batch_size,
                      callbacks=callbacks_list,
                      epochs=epochs, verbose=1)



Summary:
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 905, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 62, 903, 16)       160       
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 62, 903, 16)]     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 902, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 59, 900, 32)       4640      
_________________________________________________________________
tf_op_layer_Relu_1 (TensorFl [(None, 59, 900, 32)]     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 58, 899, 32)    

In [0]:
  print("\nHistory dict:", history.history)

  # Evaluate the model on the test data using `evaluate`
  print("Evaluating model on test data...")
  results = model.evaluate(x_test, y_test, batch_size=batch_size)
  print("Test loss:", results[0])
  print("Test acc:", results[1])